In [1]:
import numpy as np
import pandas as pd

def efficientCubic(x, v):
    """
    This function returns M, w, a, b, c, d
    where M and w are the matrix and vector in the linear system
    and a, b, c, d are the final coefficients of the linear system
    
    Note: x and v are the interpolation nodes and interpolation values
    """
    assert len(x) == len(v)
    
    #the total length
    n = len(v)
    
    #initialize empty list
    z = []
    
    M_ = np.zeros((n-1, n-1))
    
    for i in range(1, n-1):
        z_ = 6 * (np.divide(v[i + 1] - v[i], x[i+1] - x[i]) \
                 -np.divide(v[i] - v[i - 1], x[i] - x[i - 1]))
        z.append(z_)
        
        
    #calculate M
    for i in range(1, n-1):
        M_[i, i] = 2*(x[i + 1] - x[i - 1])
    for i in range(1, n-2):
        M_[i, i+1] = x[i+1] - x[i]
    for i in range(2, n-1):
        M_[i, i-1] = x[i] - x[i - 1]
        
    #ensuring M is square
    M = np.delete(M_, (0), axis = 0)
    M = np.delete(M, (0), axis = 1)
    
    #calculating w
    w = np.linalg.solve(M, z)
    
    #w0 and wn
    w = np.insert(w, 0, 0)
    w = np.append(w, 0)
    
    #a, b, c, d
    a, b, c, d = [], [], [], []
    
    q, r = [], []
    
    #calculating c and d 
    for i in range(1, n):
        c_ = np.divide(w[i - 1] * x[i] - w[i] * x[i - 1], 2*(x[i] - x[i - 1]))
        d_ = np.divide(w[i] - w[i - 1], 6*(x[i] - x[i- 1]))
        c.append(c_)
        d.append(d_)
        
    #calculating q and r 
    for i in range(1, n):
        q_ = v[i - 1] - c[i - 1] * (x[i - 1]**2) - (d[i - 1] * (x[i - 1] ** 3))
        r_ = v[i] - c[i - 1] * (x[i]**2) - d[i - 1]*(x[i]**3)
        q.append(q_)
        r.append(r_)
        
    #calculating a and b
    for i in range(1, n):
        a_ = np.divide(q[i- 1]*x[i] - r[i - 1]*x[i- 1], x[i] - x[i - 1])
        b_ = np.divide(r[i- 1] - q[i - 1], x[i] - x[i - 1])
        a.append(a_)
        b.append(b_)
        
    return M, z, w, a, b, c, d


In [2]:
#Cublic Splines...
nodes = [2, 5, 10]
T2 = [[1.69],
     [1.81],
     [1.81],
     [1.79],[1.79], [1.83], [1.81], [1.81], [1.83], [1.81], [1.82], [1.82], [1.80], [1.78], [1.79]]

T3 = [[2.58], [2.71], [2.72], [2.78], [2.77], [2.75], [2.71], [2.72], [2.76], [2.73], [2.75], [2.75], [2.73], [2.71], [2.71]]

T5 = [[3.57], [3.69], [3.70], [3.77], [3.77], [3.73], [3.72], [3.74], [3.77], [3.75], [3.77], [3.76], [3.75], [3.72], [3.71]]

T10 = [[4.63], [4.73], [4.74], [4.81], [4.80], [4.79], [4.76], [4.77], [4.80], [4.77], [4.80], [4.80], [4.78], [4.73], [4.73]]

In [4]:
from itertools import chain
T2 = list(chain.from_iterable(T2))
T3 = list(chain.from_iterable(T3))
T5 = list(chain.from_iterable(T5))
T10 = list(chain.from_iterable(T10))
df_01 = pd.DataFrame(list(zip(T2, T3, T5, T10)), columns=['T2', 'T3', 'T5', 'T10'])
df_01

,T2,T3,T5,T10
0,1.69,2.58,3.57,4.63
1,1.81,2.71,3.69,4.73
2,1.81,2.72,3.70,4.74
3,1.79,2.78,3.77,4.81
4,1.79,2.77,3.77,4.80
5,1.83,2.75,3.73,4.79
6,1.81,2.71,3.72,4.76
7,1.81,2.72,3.74,4.77
8,1.83,2.76,3.77,4.80
9,1.81,2.73,3.75,4.77


In [13]:
#Do Cublic Splines
df_01['T3_cublic'] = 0
for i in range(0, 15):
    v = [df_01['T2'][i], df_01['T5'][i], df_01['T10'][i]]
    M, z, w, a, b, c, d = efficientCubic(nodes, v)
    df_01['T3_cublic'][i] = a[0] + b[0]*3 + c[0]*(3**2) + d[0]*(3**3)
    
df_01

<ipython-input-13-34e0187f8fa9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01['T3_cublic'][i] = a[0] + b[0]*3 + c[0]*(3**2) + d[0]*(3**3)


,T2,T3,T5,T10,T3_cublic
0,1.69,2.58,3.57,4.63,2.385778
1,1.81,2.71,3.69,4.73,2.506444
2,1.81,2.72,3.70,4.74,2.510333
3,1.79,2.78,3.77,4.81,2.525333
4,1.79,2.77,3.77,4.80,2.525667
5,1.83,2.75,3.73,4.79,2.533556
6,1.81,2.71,3.72,4.76,2.518111
7,1.81,2.72,3.74,4.77,2.526222
8,1.83,2.76,3.77,4.80,2.550111
9,1.81,2.73,3.75,4.77,2.530444


In [14]:
df_01['T3_errors_cubic'] = df_01['T3'] - df_01['T3_cublic']
errorsvector = np.matrix(list(df_01.T3_errors_cubic)).T
error_cubic = np.dot(errorsvector.T, errorsvector)
error_cubic = np.sqrt(error_cubic.item())
error_cubic

0.8176751822720949